## Install requirements

In [1]:
!git clone https://github.com/wk-ff/IDN.git

Cloning into 'IDN'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 47 (delta 10), reused 9 (delta 9), pack-reused 32
Receiving objects: 100% (47/47), 9.38 KiB | 9.38 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [2]:
!pip install pynvml
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.5 MB/s eta 0:00:00


In [3]:
!pip install pytorch-ignite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.4/272.4 kB 3.8 MB/s eta 0:00:00


In [4]:
!pip install numba

In [5]:
import os
import pandas as pd
import numpy as np
from os import walk
import random
from sklearn.model_selection import KFold,StratifiedKFold

## Create Data loader

In [6]:
from torch.utils import data
import torch
import cv2

In [7]:
os.mkdir('/content/IDN/data')

In [8]:
cd /content/IDN/data

/content/IDN/data


In [9]:
!unrar x /content/drive/MyDrive/dataset/cedar.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/dataset/cedar.rar

Creating    cedar                                                     OK
Creating    cedar/images                                              OK
Extracting  cedar/images/forgeries_10_1.png                                0%  OK 
Extracting  cedar/images/forgeries_10_10.png                               0%  OK 
Extracting  cedar/images/forgeries_10_11.png                               0%  OK 
Extracting  cedar/images/forgeries_10_12.png                               0%  OK 
Extracting  cedar/images/forgeries_10_13.png                               0%  OK 
Extracting  cedar/images/forgeries_10_14.png                               0%  OK 
Extracting  cedar/images/forgeries_10_15.png                               0%  OK 
Extracting  cedar/images/forgeries_10_16.png                             

In [ ]:
class CEDARDataset(data.Dataset):
    def __init__(self, root='/content/IDN/data/cedar', train = True,
                transform = None, basic_aug = False):
        super(CEDARDataset, self).__init__()
        REF_PATH = 2
        TEST_PATH = 3
        LABEL = 4
        if train:
            path = root + '/skill/train.csv'
        else:
            path = root + '/skill/test.csv'

        self.root = root

#         with open(path, 'r') as f:
#             lines = f.readlines()
        df = pd.read_csv(path)
#         print(df.head())
        self.labels = []
        self.datas = []

        for index, row in df.iterrows():
            ref_path = '/images/' + (row['ref_path'])
            test_path = '/images/' + (row['test_path'])
            self.labels.append(int(row['label']))
            ref_img = cv2.imread(root + ref_path, 0)
            test_img = cv2.imread(root + test_path, 0)
#             print(f"path: {test_path}; type: {type(test_img)}")
            ref_img = cv2.resize(ref_img, (256, 256))
            test_img = cv2.resize(test_img, (256, 256))
#             print(ref_img.shape[0])
#             print(test_img.shape[0])
#             print(ref_img.shape[1])
#             print(test_img.shape[1])
            ref_img = ref_img.reshape(-1, ref_img.shape[0], ref_img.shape[1])
            test_img = test_img.reshape(-1, test_img.shape[0], test_img.shape[1])
            ref_test = np.concatenate((ref_img, test_img), axis = 0)
            self.datas.append(ref_test)

        # print(self.datas[0].shape)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return torch.FloatTensor(self.datas[index]), float(self.labels[index])

In [ ]:
BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 0.001

In [ ]:
train_set = CEDARDataset(train = True)
test_set = CEDARDataset(train = False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=2*BATCH_SIZE, shuffle=False)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

## Create Model

In [ ]:
cd /content/IDN

/content/IDN


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from pynvml import *
import os
from tensorboardX import SummaryWriter
import time

from dataset.dataset import dataset
from models.net import net
from loss import loss

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [ ]:
np.random.seed(128)
torch.manual_seed(1)
cuda = torch.cuda.is_available()

In [ ]:
def compute_accuracy(predicted, labels):
    for i in range(3):
        predicted[i][predicted[i] > 0.5] = 1
        predicted[i][predicted[i] <= 0.5] = 0
    predicted = predicted[0] + predicted[1] + predicted[2]

    predicted[predicted < 2] = 0
    predicted[predicted >= 2] = 1
    # print(f"predicted before view: {predicted.size()}")
    predicted = predicted.view(-1)
    print(predicted)
    # print(f"predicted after view: {predicted.size()}")
    accuracy = torch.sum(predicted == labels).item() / labels.size()[0]
    return accuracy

In [ ]:
model = net()
model = model.cuda()
criterion = loss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

## Original Skill Forgieries Dataset

In [ ]:
writer = SummaryWriter(log_dir='scalar')

if cuda:
    criterion = criterion.cuda()
iter_n = 0
t = time.strftime("%m-%d-%H-%M", time.localtime())
print(len(train_loader))
for epoch in range(1, EPOCHS + 1):
    for i, (inputs, labels) in enumerate(train_loader):
        torch.cuda.empty_cache()

        optimizer.zero_grad()

        labels = labels.float()
        if cuda:
            inputs, labels = inputs.cuda(), labels.cuda()

        predicted = model(inputs)

        loss = criterion(*predicted, labels)

        loss.backward()
        optimizer.step()

        accuracy = compute_accuracy(predicted, labels)

        writer.add_scalar(t+'/train_loss', loss.item(), iter_n)
        writer.add_scalar(t+'/train_accuracy', accuracy, iter_n)

        if i % 100 == 0:
            with torch.no_grad():
                accuracys = []
                for i_, (inputs_, labels_) in enumerate(test_loader):
                    labels_ = labels_.float()
                    if cuda:
                        inputs_, labels_ = inputs_.cuda(), labels_.cuda()
                    predicted_ = model(inputs_)
                    accuracys.append(compute_accuracy(predicted_, labels_))
                accuracy_ = sum(accuracys) / len(accuracys)
                writer.add_scalar(t+'/test_accuracy', accuracy_, iter_n)
            print('test loss:{:.6f}'.format(accuracy_))

        iter_n += 1

        if i == 500:
            torch.save(model.state_dict(), 'model.pth')

        if i % 10 == 0:
            print('Epoch[{}/{}], iter {}, loss:{:.6f}, accuracy:{}'.format(epoch, EPOCHS, i, loss.item(), accuracy))

writer.close()

1332
test loss:1.000000
Epoch[1/2], iter 0, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 10, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 20, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 30, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 40, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 50, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 60, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 70, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 80, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 90, loss:0.000000, accuracy:1.0
test loss:1.000000
Epoch[1/2], iter 100, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 110, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 120, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 130, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 140, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 150, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 160, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 170, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 180, loss:0.000000, accuracy:1.0
Epoch[1/2], iter 1

## Duong's Random Forgieries Dataset

In [10]:
cd /content/IDN

/content/IDN


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from pynvml import *
import os
from tensorboardX import SummaryWriter
import time
import pandas as pd

from dataset.dataset import dataset
from models.net import net
from loss import loss
from torch.utils import data
import torch
import cv2
import numpy as np

In [12]:
import numpy as np
import random
import pandas as pd
import os, glob, shutil

if __name__ == "__main__":
    np.random.seed(42)
    random.seed(42)

    rows = []
    user_train = np.arange(1, 44) # [2,3,5,8,9,10,12,13,15,16,17,18,22,30]
    user_test = np.arange(44, 56)

    for user1 in user_train:
        for img_id1 in range(1, 25):
            for img_id2 in range(img_id1, 25):
                rows.append('original_{0}_{1}.png original_{0}_{2}.png 1 2'.format(user1, img_id1, img_id2).split())

        org_forg = [(img_id1, img_id2) for img_id1 in range(1, 25) for img_id2 in range(1, 25)]
        org_forg = random.choices(org_forg, k=len(org_forg) // 4)
        for (img_id1, img_id2) in org_forg:
            rows.append('original_{0}_{1}.png forgeries_{0}_{2}.png 0 1'.format(user1, img_id1, img_id2).split())

        for img_id1 in range(1, 25):
            user2_img_id2 = []
            for user2 in user_train:
                if user2 == user1:
                    continue
                for img_id2 in range(1, 25):
                    user2_img_id2.append((user2, img_id2))

            for user2, img_id2 in random.choices(user2_img_id2, k=6):
                rows.append('original_{0}_{1}.png original_{2}_{3}.png 0 0'.format(user1, img_id1, user2, img_id2).split())

    train_df = pd.DataFrame(rows, columns=['image1', 'image2', 'label', 'level'])
    train_df.to_csv("train_v2.csv", index=False)
    print(train_df[['label', 'level']].value_counts())

    rows = []
    for user1 in user_test:
        for img_id1 in range(1, 25):
            for img_id2 in range(img_id1, 25):
                rows.append('original_{0}_{1}.png original_{0}_{2}.png 1 2'.format(user1, img_id1, img_id2).split())
        org_forg = [(img_id1, img_id2) for img_id1 in range(1, 25) for img_id2 in range(1, 25)]
        org_forg = random.choices(org_forg, k=len(org_forg) // 4)
        for (img_id1, img_id2) in org_forg:
            rows.append('original_{0}_{1}.png forgeries_{0}_{2}.png 0 1'.format(user1, img_id1, img_id2).split())

        for img_id1 in range(1, 25):
            user2_img_id2 = []
            for user2 in user_test:
                if user2 == user1:
                    continue
                for img_id2 in range(1, 25):
                    user2_img_id2.append((user2, img_id2))

            for user2, img_id2 in random.choices(user2_img_id2, k=6):
                rows.append('original_{0}_{1}.png original_{2}_{3}.png 0 0'.format(user1, img_id1, user2, img_id2).split())
    val_df = pd.DataFrame(rows, columns=['image1', 'image2', 'label', 'level'])
    val_df.to_csv("val_v2.csv", index=False)
    print(val_df[['label', 'level']].value_counts())
  # Dataset size: 32340

label  level
1      2        12900
0      0         6192
       1         6192
dtype: int64
label  level
1      2        3600
0      0        1728
       1        1728
dtype: int64


In [ ]:
class CEDARRandomDataset(data.Dataset):
    def __init__(self, root='/content/IDN/data/cedar', train = True,
                transform = None, basic_aug = False):
        super(CEDARRandomDataset, self).__init__()
        if train:
            path = '/content/IDN/train_v2.csv'
        else:
            path = '/content/IDN/val_v2.csv'

        self.root = root

#         with open(path, 'r') as f:
#             lines = f.readlines()
        df = pd.read_csv(path)
#         print(df.head())
        self.labels = []
        self.datas = []

        for index, row in df.iterrows():
            ref_path = '/images/' + (row['image1'])
            test_path = '/images/' + (row['image2'])
            self.labels.append(int(row['label']))
            ref_img = cv2.imread(root + ref_path, 0)
            test_img = cv2.imread(root + test_path, 0)
#             print(f"path: {test_path}; type: {type(test_img)}")
            ref_img = cv2.resize(ref_img, (256, 256))
            test_img = cv2.resize(test_img, (256, 256))
#             print(ref_img.shape[0])
#             print(test_img.shape[0])
#             print(ref_img.shape[1])
#             print(test_img.shape[1])
            ref_img = ref_img.reshape(-1, ref_img.shape[0], ref_img.shape[1])
            test_img = test_img.reshape(-1, test_img.shape[0], test_img.shape[1])
            ref_test = np.concatenate((ref_img, test_img), axis = 0)
            self.datas.append(ref_test)

        # print(self.datas[0].shape)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return torch.FloatTensor(np.array(self.datas[index])), float(self.labels[index])

In [13]:
def get_datas(root, ref_path, test_path):
      ref_img = cv2.imread(root + ref_path, 0)
      test_img = cv2.imread(root + test_path, 0)
      # ref_img = cv2.cvtColor(ref_img, cv2.COLOR_RGB2GRAY )
      # test_img = cv2.cvtColor(test_img, cv2.COLOR_RGB2GRAY )
      ref_img = cv2.resize(ref_img, (256, 256))
      test_img = cv2.resize(test_img, (256, 256))
      ref_img = ref_img.reshape(-1, ref_img.shape[0], ref_img.shape[1])
      test_img = test_img.reshape(-1, test_img.shape[0], test_img.shape[1])
      ref_test = np.concatenate((ref_img, test_img), axis = 0)
      return ref_test

In [14]:
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool

In [ ]:
class CEDARRandomDataset(data.Dataset):
    def __init__(self, root='/content/IDN/data/cedar', train = True,
                transform = None, basic_aug = False):
        super(CEDARRandomDataset, self).__init__()
        REF_PATH = 2
        TEST_PATH = 3
        LABEL = 4
        if train:
            path = '/content/IDN/train_v2.csv'
        else:
            path = '/content/IDN/val_v2.csv'

        self.root = root

        df = pd.read_csv(path)
        self.labels = []
        self.datas = []

        for index, row in df.iterrows():
            ref_path = '/images/' + (row['image1'])
            test_path = '/images/' + (row['image2'])
            self.labels.append(int(row['label']))
            with ThreadPool(processes = 4) as p:
              ref_test = (p.starmap(get_datas, [(root, ref_path, test_path)]))
              self.datas.append(ref_test)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return torch.FloatTensor(np.array(self.datas[index])), float(self.labels[index])

In [15]:
np.random.seed(128)
torch.manual_seed(1)
cuda = torch.cuda.is_available()

In [16]:
BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 0.001

In [ ]:
train_set = CEDARRandomDataset(train = True)
test_set = CEDARRandomDataset(train = False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=2*BATCH_SIZE, num_workers = 2, shuffle=False)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, num_workers = 2, shuffle=True)

In [17]:
model = net()
model = model.cuda()
criterion = loss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
from collections import OrderedDict

import torch
from torch import nn, optim

from ignite.engine import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.utils import *
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *

# create default evaluator for doctests

def eval_step(engine, batch):
    return batch

default_evaluator = Engine(eval_step)

In [ ]:
def binary_one_hot_output_transform(output):
    y_pred, y = output
    y_pred = torch.sigmoid(y_pred).round().long()
    y_pred = ignite.utils.to_onehot(y_pred, 2)
    y = y.long()
    return y_pred, y

In [ ]:
from sklearn.metrics import confusion_matrix
from ignite.metrics.confusion_matrix import ConfusionMatrix
def compute_far(predicted, labels):
    for i in range(3):
        predicted[i][predicted[i] > 0.5] = 1
        predicted[i][predicted[i] <= 0.5] = 0
    predicted = predicted[0] + predicted[1] + predicted[2]

    predicted[predicted < 2] = 0
    predicted[predicted >= 2] = 1
    predicted = predicted.view(-1)
    cfs_matrix = confusion_matrix(predicted.cpu().detach().numpy(), labels.cpu().detach().numpy(), labels=[True, False])
    # print(cfs_matrix)
    # accuracy = torch.sum(predicted == labels).item() / labels.size()[0]
    if(cfs_matrix[1][0] == 0 and cfs_matrix[1][1] == 0):
      far = 0
    else:
      far = float(cfs_matrix[1][0] / (cfs_matrix[1][0] + cfs_matrix[1][1]) * 100)
    # print(far)
    return far

In [ ]:
from sklearn.metrics import confusion_matrix
from ignite.metrics.confusion_matrix import ConfusionMatrix
def compute_frr(predicted, labels):
    for i in range(3):
        predicted[i][predicted[i] > 0.5] = 1
        predicted[i][predicted[i] <= 0.5] = 0
    predicted = predicted[0] + predicted[1] + predicted[2]

    predicted[predicted < 2] = 0
    predicted[predicted >= 2] = 1
    predicted = predicted.view(-1)
    cfs_matrix = confusion_matrix(predicted.cpu().detach().numpy(), labels.cpu().detach().numpy(), labels=[True, False])
    if(cfs_matrix[0][0] == 0 and cfs_matrix[0][1] == 0):
      frr = 0
    else:
      frr = float(cfs_matrix[0][1] / (cfs_matrix[0][1] + cfs_matrix[0][0]) * 100)
    return frr

In [ ]:
def compute_far_test(predicted, labels):
    for i in range(3):
        predicted[i][predicted[i] > 0.5] = 1
        predicted[i][predicted[i] <= 0.5] = 0
    predicted = predicted[0] + predicted[1] + predicted[2]

    predicted[predicted < 2] = 0
    predicted[predicted >= 2] = 1
    cfs_matrix = confusion_matrix(predicted.cpu().numpy(), labels.cpu().numpy(), labels=[True, False])
    if(cfs_matrix[1][0] == 0 and cfs_matrix[1][1] == 0):
      far = 0
    else:
      far = float(cfs_matrix[1][0] / (cfs_matrix[1][0] + cfs_matrix[1][1]) * 100)
    return far

In [ ]:
def compute_frr_test(predicted, labels):
    for i in range(3):
        predicted[i][predicted[i] > 0.5] = 1
        predicted[i][predicted[i] <= 0.5] = 0
    predicted = predicted[0] + predicted[1] + predicted[2]

    predicted[predicted < 2] = 0
    predicted[predicted >= 2] = 1
    cfs_matrix = confusion_matrix(predicted.cpu().numpy(), labels.cpu().numpy(), labels=[True, False])
    if(cfs_matrix[0][0] == 0 and cfs_matrix[0][1] == 0):
      frr = 0
    else:
      frr = float(cfs_matrix[0][1] / (cfs_matrix[0][1] + cfs_matrix[0][0]) * 100)
    return frr

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
writer = SummaryWriter(log_dir='scalar')

if cuda:
    criterion = criterion.cuda()
iter_n = 0
t = time.strftime("%m-%d-%H-%M", time.localtime())
print(len(train_loader))
for epoch in range(1, EPOCHS + 1):
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        torch.cuda.empty_cache()

        optimizer.zero_grad()

        labels = labels.float()
        if cuda:
            inputs, labels = inputs.cuda(), labels.cuda()
        inputs = np.squeeze(inputs)
        predicted = model(inputs)
        # print(inputs)
        # print(predicted)
        loss = criterion(*predicted, labels)

        loss.backward()
        optimizer.step()

        far_score = compute_far(predicted, labels)
        frr_score = compute_frr(predicted, labels)

        writer.add_scalar(t+'/train_loss', loss.item(), iter_n)
        writer.add_scalar(t+'/train_far', far_score, iter_n)

        if i % 200 == 0:
            with torch.no_grad():
                far_scores = []
                frr_scores = []
                model.eval()
                for i_, (inputs_, labels_) in enumerate(test_loader):
                    labels_ = labels_.float()
                    # print(labels_)
                    if cuda:
                        inputs_, labels_ = inputs_.cuda(), labels_.cuda()
                    inputs_ = np.squeeze(inputs_)
                    predicted_ = model(inputs_)
                    # print(len(inputs_))
                    # print(len(predicted_))
                    far_scores.append(compute_far_test(predicted_, labels_))
                    frr_scores.append(compute_frr_test(predicted_, labels_))
                far_scores_ = sum(far_scores) / len(far_scores)
                frr_scores_ = sum(frr_scores) / len(frr_scores)
                writer.add_scalar(t+'/test_accuracy', far_scores_, iter_n)
            print('test far:{:.6f}'.format(far_scores_))
            print('test frr:{:.6f}'.format(frr_scores_))
            if(far_scores_ < 10):
              print("Saving best model")
              torch.save(model.state_dict(), 'best.pth')

        iter_n += 1

        if i == 500:
            torch.save(model.state_dict(), 'model.pth')

        if i % 10 == 0:
            print('Epoch[{}/{}], iter {}, loss:{:.6f}, FAR:{}, FRR:{}'.format(epoch, EPOCHS, i, loss.item(), far_score, frr_score))

writer.close()

791
test far:50.675676
test frr:0.000000
Epoch[1/2], iter 0, loss:0.690880, FAR:43.75, FRR:0
Epoch[1/2], iter 10, loss:0.742115, FAR:0, FRR:71.875
Epoch[1/2], iter 20, loss:0.714341, FAR:0, FRR:62.5
Epoch[1/2], iter 30, loss:0.687153, FAR:40.625, FRR:0
Epoch[1/2], iter 40, loss:0.681410, FAR:40.625, FRR:0
Epoch[1/2], iter 50, loss:0.617384, FAR:0.0, FRR:35.0
Epoch[1/2], iter 60, loss:0.728992, FAR:0, FRR:62.5
Epoch[1/2], iter 70, loss:0.649320, FAR:0.0, FRR:35.714285714285715
Epoch[1/2], iter 80, loss:0.548565, FAR:34.375, FRR:0
Epoch[1/2], iter 90, loss:0.579725, FAR:70.96774193548387, FRR:0.0
test far:36.574769
test frr:30.303848
Epoch[1/2], iter 100, loss:0.457503, FAR:0.0, FRR:33.33333333333333
Epoch[1/2], iter 110, loss:0.617250, FAR:0.0, FRR:44.44444444444444
Epoch[1/2], iter 120, loss:0.544167, FAR:47.82608695652174, FRR:33.33333333333333
Epoch[1/2], iter 130, loss:0.571858, FAR:0.0, FRR:46.15384615384615
Epoch[1/2], iter 140, loss:0.435455, FAR:0.0, FRR:21.73913043478261
Epoch[

In [ ]:
/content/IDN/scalar/events.out.tfevents.1700642403.3a20e9ec7d58

## K-Fold Cross Validation (k = 11)

In [18]:
cd /content/IDN

/content/IDN


In [19]:
train_path = '/content/IDN/train_v2.csv'
test_path = '/content/IDN/val_v2.csv'
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_temp = df_train.merge(df_test, how = 'outer')

In [20]:
len(df_test)

7056

In [21]:
len(df_temp)

32340

In [22]:
# Merge train and test data from 2 csv files
class CEDARRandomDataset(data.Dataset):
    def __init__(self, root='/content/IDN/data/cedar',
                transform = None, basic_aug = False):
        super(CEDARRandomDataset, self).__init__()
        train_path = '/content/IDN/train_v2.csv'
        test_path = '/content/IDN/val_v2.csv'

        self.root = root

        df_train = pd.read_csv(train_path)
        df_test = pd.read_csv(test_path)
        df_total = df_train.merge(df_test, how = 'outer')
        self.labels = []
        self.datas = []

        for index, row in df_total.iterrows():
            ref_path = '/images/' + (row['image1'])
            test_path = '/images/' + (row['image2'])
            self.labels.append(int(row['label']))
            with ThreadPool(processes = 4) as p:
              ref_test = (p.starmap(get_datas, [(root, ref_path, test_path)]))
              self.datas.append(ref_test)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return torch.FloatTensor(np.array(self.datas[index])), float(self.labels[index])

In [23]:
dataset_total = CEDARRandomDataset()

In [37]:
from sklearn.metrics import confusion_matrix
from ignite.metrics.confusion_matrix import ConfusionMatrix
def compute_metrics(predicted, labels, is_train = True):
    for i in range(3):
        predicted[i][predicted[i] > 0.5] = 1
        predicted[i][predicted[i] <= 0.5] = 0
    predicted = predicted[0] + predicted[1] + predicted[2]

    predicted[predicted < 2] = 0
    predicted[predicted >= 2] = 1
    predicted = predicted.view(-1)
    if(is_train):
      cfs_matrix = confusion_matrix(predicted.cpu().detach().numpy(), labels.cpu().detach().numpy(), labels=[True, False])
    else:
      cfs_matrix = confusion_matrix(predicted.cpu().numpy(), labels.cpu().numpy(), labels=[True, False])
    if(cfs_matrix[0][0] == 0 and cfs_matrix[0][1] == 0):
      frr = 0
    else:
      frr = float(cfs_matrix[0][1] / (cfs_matrix[0][1] + cfs_matrix[0][0]) * 100)
    if(cfs_matrix[1][0] == 0 and cfs_matrix[1][1] == 0):
      far = 0
    else:
      far = float(cfs_matrix[1][0] / (cfs_matrix[1][0] + cfs_matrix[1][1]) * 100)
    accuracy = float((cfs_matrix[0][0] + cfs_matrix[1][1]) / (cfs_matrix[0][1] + cfs_matrix[1][0] + cfs_matrix[0][0] + cfs_matrix[1][1]))
    return far, frr, accuracy

In [44]:
from time import time
def train_idn(model, criterion, optimizer, train_loader, test_loader, epochs = 1):

  if cuda:
      criterion = criterion.cuda()
  iter_n = 0
  # t = time.strftime("%m-%d-%H-%M", time.localtime())
  best_model = [100, 100, 0]
  for epoch in range(1, epochs + 1):
      start_time = time()
      model.train()
      for i, (inputs, labels) in enumerate(train_loader):
          torch.cuda.empty_cache()

          optimizer.zero_grad()

          labels = labels.float()
          if cuda:
              inputs, labels = inputs.cuda(), labels.cuda()
          inputs = np.squeeze(inputs)
          predicted = model(inputs)
          # print(inputs)
          # print(predicted)
          loss = criterion(*predicted, labels)

          loss.backward()
          optimizer.step()
          far_score, frr_score, accuracy_score = compute_metrics(predicted, labels, is_train = True)

          if i % 200 == 0:
              with torch.no_grad():
                  far_scores = []
                  frr_scores = []
                  accuracy_scores = []
                  model.eval()
                  for i_, (inputs_, labels_) in enumerate(test_loader):
                      labels_ = labels_.float()
                      # print(labels_)
                      if cuda:
                          inputs_, labels_ = inputs_.cuda(), labels_.cuda()
                      inputs_ = np.squeeze(inputs_)
                      predicted_ = model(inputs_)
                      # print(len(inputs_))
                      # print(len(predicted_))
                      far, frr, accuracy = compute_metrics(predicted_, labels_, is_train = False)
                      far_scores.append(far)
                      frr_scores.append(frr)
                      accuracy_scores.append(accuracy)
                  far_scores_ = sum(far_scores) / len(far_scores)
                  frr_scores_ = sum(frr_scores) / len(frr_scores)
                  accuracy_scores_ = sum(accuracy_scores) / len(accuracy_scores)
                  # print(f"far_scores_: {type(far_scores_)}")
                  # print(f"best model: {type(best_model[0])}")
                  if(float(far_scores_) < float(best_model[0]) and float(frr_scores_) < float(best_model[1]) and float(accuracy_scores_) > float(best_model[2])):
                    best_model[0] = far_scores_
                    best_model[1] = frr_scores_
                    best_model[2] = accuracy_scores_
              print('test far:{:.6f}'.format(far_scores_))
              print('test frr:{:.6f}'.format(frr_scores_))
              print('test acc:{:.6f}'.format(accuracy_scores_))
              if(far_scores_ < 10):
                print("Saving best model")
                torch.save(model.state_dict(), 'best.pth')

          iter_n += 1

          if i == 500:
              torch.save(model.state_dict(), 'model.pth')

          if i % 10 == 0:
              print('Epoch[{}/{}], iter {}, loss:{:.6f}, FAR:{}, FRR:{}, ACC:{}'.format(epoch, epochs, i, loss.item(), far_score, frr_score, accuracy_score))
      train_time = (time() - start_time) / 60
      print(f"Train time: {train_time} min")

  return best_model[0], best_model[1], best_model[2]

In [38]:
# define a cross validation function
def crossvalid(model=None,criterion=None,optimizer=None,dataset=None,k_fold=5):

    # train_accuracy = pd.Series()
    # train_far = pd.Series()
    # train_frr = pd.Series()
    val_accuracy = pd.Series(dtype='float64')
    val_far = pd.Series(dtype='float64')
    val_frr = pd.Series(dtype='float64')

    total_size = len(dataset)
    fraction = 1/k_fold
    seg = int(total_size * fraction)
    # tr:train,val:valid; r:right,l:left;  eg: trrr: right index of right side train subset
    # index: [trll,trlr],[vall,valr],[trrl,trrr]
    for i in range(k_fold):
        trll = 0
        trlr = i * seg
        vall = trlr
        valr = i * seg + seg
        trrl = valr
        trrr = total_size
        # msg
        print(f"Fold {i + 1}:")
        print("train indices: [%d,%d),[%d,%d), test indices: [%d,%d)"
              % (trll,trlr,trrl,trrr,vall,valr))

        train_left_indices = list(range(trll,trlr))
        train_right_indices = list(range(trrl,trrr))

        train_indices = train_left_indices + train_right_indices
        val_indices = list(range(vall,valr))

        train_set = torch.utils.data.dataset.Subset(dataset,train_indices)
        val_set = torch.utils.data.dataset.Subset(dataset,val_indices)

#         print(len(train_set),len(val_set))
#         print()
        train_loader = torch.utils.data.DataLoader(train_set, batch_size=32,
                                          shuffle=True, num_workers=2)
        val_loader = torch.utils.data.DataLoader(val_set, batch_size=32,
                                          shuffle=True, num_workers=2)
        # train_acc = train(res_model,criterion,optimizer,train_loader,epoch=1)
        # train_score.at[i] = train_acc
        val_far_fold, val_frr_fold ,val_acc_fold = train_idn(model, criterion, optimizer, train_loader, val_loader, epochs = 1)
        # val_acc = valid(res_model,criterion,optimizer,val_loader)
        val_far.at[i] = val_far_fold
        val_frr.at[i] = val_frr_fold
        val_accuracy.at[i] = val_acc_fold
        print(f"Fold {i}, validate accuracy: {val_acc_fold}, validate FAR: {val_far_fold}, validate FRR: {val_frr_fold}")
    return val_accuracy, val_far, val_frr


# train_score,val_score = crossvalid(res_model,criterion,optimizer,dataset=tiny_dataset)

In [ ]:
val_accuracy, val_far, val_frr = crossvalid(model ,criterion, optimizer, dataset = dataset_total, k_fold = 11)

Fold 1:
train indices: [0,0),[2940,32340), test indices: [0,2940)
test far:0.000000
test frr:48.980978
test acc:0.510190
Saving best model
Epoch[1/2], iter 0, loss:0.633050, FAR:0, FRR:34.375, ACC:0.65625
Epoch[1/2], iter 10, loss:0.652434, FAR:0, FRR:59.375, ACC:0.40625
Epoch[1/2], iter 20, loss:0.647905, FAR:0, FRR:50.0, ACC:0.5
Epoch[1/2], iter 30, loss:0.663028, FAR:0, FRR:50.0, ACC:0.5
Epoch[1/2], iter 40, loss:0.647126, FAR:0, FRR:53.125, ACC:0.46875
Epoch[1/2], iter 50, loss:0.646529, FAR:0, FRR:50.0, ACC:0.5
Epoch[1/2], iter 60, loss:0.621867, FAR:0, FRR:59.375, ACC:0.40625
Epoch[1/2], iter 70, loss:0.622388, FAR:0, FRR:53.125, ACC:0.46875
Epoch[1/2], iter 80, loss:0.628166, FAR:0, FRR:37.5, ACC:0.625
Epoch[1/2], iter 90, loss:0.666746, FAR:0, FRR:53.125, ACC:0.46875
Epoch[1/2], iter 100, loss:0.640236, FAR:0, FRR:62.5, ACC:0.375
Epoch[1/2], iter 110, loss:0.621684, FAR:0, FRR:37.5, ACC:0.625
Epoch[1/2], iter 120, loss:0.657248, FAR:0, FRR:40.625, ACC:0.59375
Epoch[1/2], iter 1